In [9]:
import pandas as pd
import numpy as np
import xmltodict
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [25]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [26]:
# Load the webpage
driver.get("https://carsandbids.com/auctions/rxV7kL4j/2002-porsche-911-carrera-coupe")

In [4]:
## retrieve Usernames of commenters ##
# Scroll to bottom of page to dynamically load everything
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True
time.sleep(3)

# find load more comments button and click
eop = False
while(eop == False):
    try:
        load_more = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "load-more"))
        )
    except:
        # if no more to load, stop scrolling
        print("end of page reached")
        eop = True
    else:
        # if more to load, click and scroll
        load_more.click()
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match=False
        while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
        time.sleep(3)

end of page reached


In [7]:
# Get comment section
try:
    main = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "comments"))
    )
except:
    driver.quit()

# get comment thread
thread = main.find_element(By.CLASS_NAME, 'thread')
# get each element. could be comment, flagged comment, bid, etc.
comments = thread.find_elements(By.XPATH, '*')
# print user of each comment
thread_df = pd.DataFrame(columns = ['position', 'user', 'comment', 'bid'])
position = 1
for comment in comments:
    # get username
    try:
        username = comment.find_element(By.CLASS_NAME, 'user').text
    except:
        username = None
    # get message
    try: 
        message = comment.find_element(By.CLASS_NAME, 'message').text
    except:
        message = None
    # get bid
    try:
        bid = comment.find_element(By.CLASS_NAME, 'bid-value').text
    except:
        bid = None
    
    thread_df.loc[len(thread_df.index)] = [position, username, message, bid] 
    position += 1


In [28]:
head = driver.find_element(By.CLASS_NAME, 'auction-title')

In [8]:
print(thread_df)

    position              user  \
0          1       JohnnyRingo   
1          2         CressiDuh   
2          3  Stick_Shift_Nick   
3          4          davidnyc   
4          5           avrus96   
..       ...               ...   
117      118          msitayem   
118      119    burtmacklinfbi   
119      120          AndrewRS   
120      121          AndrewRS   
121      122          pootatoo   

                                               comment     bid  
0                   Re: davidnyc\nThanks for the info.    None  
1                                        Thank you all    None  
2                     Great buy! Enjoy in good health!    None  
3      Re: JohnnyRingo\nChavis Performance Engineering    None  
4      Congrats to @CressiDuh! I think you'll love it.    None  
..                                                 ...     ...  
117                                               None  $1,234  
118  Impressive swap with what looks to be working ...    None  
119  R

In [13]:
quick_facts = driver.find_element(By.CLASS_NAME, 'quick-facts')

In [23]:
dl = quick_facts.find_element(By.XPATH, '//dl/dd[2]/a')

In [24]:
dl.text

'911'